# import packages and connect to google drive

In [ ]:
!pip install mat73

In [ ]:
!pip install vitaldb

In [ ]:
# import the right packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import einops as eo # Added
import pathlib as pl # Added

from mat73 import loadmat

from torch.utils.data import ConcatDataset

import wfdb
import ast
import pickle
import gzip
import random
import urllib.request
import shutil
import ast
from multiprocessing import Pool, cpu_count
# from tqdm import tqdm # Removed redundant import
import os
import itertools
import matplotlib.pyplot as plt
import gdown
import json
from collections import defaultdict

import matplotlib.cm as cm
from matplotlib import collections  as mc
from matplotlib import animation
%matplotlib inline

from scipy.stats import norm
from scipy.stats import entropy
from scipy.ndimage import uniform_filter1d
from scipy.signal import resample
from datetime import datetime


#transformer:
from transformers import AutoTokenizer, AutoModel



from PIL import Image
from time import time as timer
import umap

from IPython.display import HTML
from IPython.display import Audio
import IPython

from functools import partial
from collections import Counter

from tqdm.auto import tqdm # Corrected import to get the callable tqdm function

import torch
from torchvision import datasets, transforms
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F


import sys
is_colab = 'google.colab' in sys.modules

from google.colab import drive
drive.mount('/content/drive')

from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler

import glob
import vitaldb

import scipy.io as sio

from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr

# Load data (do not rerun)

https://github.com/pulselabteam/PulseDB

https://www.frontiersin.org/journals/digital-health/articles/10.3389/fdgth.2022.1090854/full

it seems that I have shared the access to the file containing the segmented data in google drive. data are now at this link: https://drive.google.com/drive/folders/1QZ-Z-C9MoLk1S9MovF3Tt6NjAEGbSJI3

there is no need to download file

!!!!! for the shortcut: first ask to share the file. then the file will appeared in shared file in Gdrive. then create a shortcut to put it where you want it to be.

In [ ]:
# Navigate to the folder
data_dir = '/content/drive/MyDrive/CAS_FinalProject/PulseDB_Vital' # link to the shortcut
save_dir = '/content/drive/MyDrive/CAS_FinalProject/CAS_FinalProject_RAW/Vital1'

############################################### careful: the data at this link contain only age and sex demographic info

# data_dir = '/content/drive/MyDrive/Colab_Notebooks/CAS_FinalProject/PulseDB_Vital'# Colab_Notebook changed to remove the space; filed moved to /content/drive/MyDrive/CAS_FinalProject/CAS_FinalProject_RAW/PulseDB_Vital
# save_dir = '/content/drive/MyDrive/Colab_Notebooks/CAS_FinalProject/Vital' ##### Note that there is Colab_Notebook in the file path/ data moved to /content/drive/MyDrive/CAS_FinalProject/CAS_FinalProject_RAW/Vital
os.makedirs(save_dir, exist_ok=True)

# Progress tracking file
progress_file = os.path.join(save_dir, 'processing_progress.json')

# List all .mat files
mat_files = sorted(glob.glob(os.path.join(data_dir, '*.mat')))
print(f"Found {len(mat_files)} .mat files\n")


def Build_Dataset(file_path, FieldName='Subj_Wins'):
    """Load PulseDB segment file and extract signals and demographics"""
    Data = loadmat(file_path)
    SubjData = Data[FieldName]

    # Access and squeeze signals
    ECG_Record_F = np.squeeze(np.array(SubjData['ECG_Record_F']))
    PPG_Record_F = np.squeeze(np.array(SubjData['PPG_Record_F']))
    T = np.squeeze(np.array(SubjData['T']))

    # Ensure 2D
    if ECG_Record_F.ndim == 1:
        ECG_Record_F = ECG_Record_F.reshape(1, -1)
        PPG_Record_F = PPG_Record_F.reshape(1, -1)
        T = T.reshape(1, -1)

    # Stack signals
    n_segments = ECG_Record_F.shape[0]
    signal_length = ECG_Record_F.shape[1]
    Signals = np.zeros((n_segments, 3, signal_length))
    Signals[:, 0, :] = ECG_Record_F
    Signals[:, 1, :] = PPG_Record_F
    Signals[:, 2, :] = T

    # Demographics
    Age = np.array(SubjData['Age']).flatten()
    Gender = np.array(SubjData['Gender']).flatten()

    if Gender.dtype.kind in ['U', 'S']:
        Gender = np.array([1.0 if 'M' in str(g).upper() else 0.0 for g in Gender])
    else:
        Gender = (Gender == 'M').astype(float)

    Height = np.array(SubjData['Height']).flatten()
    Weight = np.array(SubjData['Weight']).flatten()

    if Age.size == 1 and n_segments > 1:
        Age = np.repeat(Age, n_segments)
        Gender = np.repeat(Gender, n_segments)
        Height = np.repeat(Height, n_segments)
        Weight = np.repeat(Weight, n_segments)

    Demographics = np.stack((Age, Gender, Height, Weight), axis=1)

    return Signals, Demographics


def load_progress():
    """Load processing progress from file"""
    if os.path.exists(progress_file):
        with open(progress_file, 'r') as f:
            progress = json.load(f)
        print(f"Resuming from batch {progress['last_batch'] + 1}")
        print(f"Already processed: {progress['processed_files']} files")
        return progress
    else:
        return {'last_batch': -1, 'processed_files': 0, 'processed_filenames': []}


def save_progress(batch_num, processed_count, processed_filenames):
    """Save processing progress to file"""
    progress = {
        'last_batch': batch_num,
        'processed_files': processed_count,
        'processed_filenames': processed_filenames
    }
    with open(progress_file, 'w') as f:
        json.dump(progress, f)


def save_batch(batch_num, signals, demographics, subject_ids):
    """Save a batch of processed data"""
    batch_path = os.path.join(save_dir, f'PulseDB_Vital_Batch_{batch_num:04d}.npz')
    np.savez_compressed(batch_path,
                        Signals=signals,
                        Demographics=demographics,
                        SubjectIDs=subject_ids)
    print(f"✅ Saved batch {batch_num} to: {batch_path}")
    return batch_path


def load_all_batches():
    """Load all previously saved batches and combine them"""
    batch_files = sorted(glob.glob(os.path.join(save_dir, 'PulseDB_Vital_Batch_*.npz')))

    if not batch_files:
        return None, None, None

    print(f"\nLoading {len(batch_files)} existing batches...")
    all_signals = []
    all_demographics = []
    all_subject_ids = []

    for batch_file in batch_files:
        data = np.load(batch_file)
        all_signals.append(data['Signals'])
        all_demographics.append(data['Demographics'])
        all_subject_ids.append(data['SubjectIDs']) ############################ changed

    combined_signals = np.concatenate(all_signals, axis=0)
    combined_demographics = np.concatenate(all_demographics, axis=0)
    combined_subject_ids = np.concatenate(all_subject_ids, axis=0) ############################## changed

    print(f"Loaded {len(batch_files)} batches with {combined_signals.shape[0]} total segments")

    return combined_signals, combined_demographics, combined_subject_ids


# Load progress
progress = load_progress()
start_idx = progress['processed_files']
processed_filenames = set(progress['processed_filenames'])

# Skip already processed files
remaining_files = [f for f in mat_files if os.path.basename(f) not in processed_filenames]
print(f"Remaining files to process: {len(remaining_files)}")

# Process files in batches of 50
BATCH_SIZE = 50
batch_num = progress['last_batch'] + 1

batch_signals = []
batch_demographics = []
batch_subject_ids = []
total_processed = start_idx

print(f"\nStarting processing from file {start_idx + 1}...")

for idx, mat_file in enumerate(remaining_files):
    try:
        Signals, Demographics = Build_Dataset(mat_file)

        batch_signals.append(Signals)
        batch_demographics.append(Demographics)

        # Extract subject ID from filename
        subject_id = os.path.splitext(os.path.basename(mat_file))[0]
        batch_subject_ids.extend([subject_id] * Signals.shape[0])

        processed_filenames.add(os.path.basename(mat_file))
        total_processed += 1

        # Save every 50 subjects
        if len(batch_signals) >= BATCH_SIZE:
            print(f"\nSaving batch {batch_num} ({len(batch_signals)} subjects)...")

            # Concatenate batch data
            batch_signals_concat = np.concatenate(batch_signals, axis=0)
            batch_demographics_concat = np.concatenate(batch_demographics, axis=0)
            batch_subject_ids_array = np.array(batch_subject_ids)

            # Save batch
            save_batch(batch_num, batch_signals_concat, batch_demographics_concat, batch_subject_ids_array)

            # Update progress
            save_progress(batch_num, total_processed, list(processed_filenames))

            # Reset batch lists
            batch_signals = []
            batch_demographics = []
            batch_subject_ids = []
            batch_num += 1

        # Progress update
        if (idx + 1) % 10 == 0:
            print(f"Processed {idx + 1}/{len(remaining_files)} remaining files (Total: {total_processed}/{len(mat_files)})")

    except Exception as e:
        print(f"Error processing {os.path.basename(mat_file)}: {e}")
        continue


# Save remaining subjects (less than 50)
if len(batch_signals) > 0:
    print(f"\nSaving final batch {batch_num} ({len(batch_signals)} subjects)...")
    batch_signals_concat = np.concatenate(batch_signals, axis=0)
    batch_demographics_concat = np.concatenate(batch_demographics, axis=0)
    batch_subject_ids_array = np.array(batch_subject_ids)

    save_batch(batch_num, batch_signals_concat, batch_demographics_concat, batch_subject_ids_array)
    save_progress(batch_num, total_processed, list(processed_filenames))


In [ ]:
# Path to your npz file
file_path = '/content/drive/MyDrive/CAS_FinalProject/CAS_FinalProject_RAW/Vital1/PulseDB_Vital_Batch_0000.npz'

# Load the npz file
data = np.load(file_path, allow_pickle=True)

if 'Demographics' in data.files:
    demographics = data['Demographics']
    print("Demographics shape:", demographics.shape)
    print("Demographics dtype:", demographics.dtype)


    print("\nFirst 5 rows of Demographics:")
    print(demographics[:5])
else:
    print("Demographics key not found in the file.")

if 'SubjectIDs' in data.files:
    subject_ids = data['SubjectIDs']
    print(f"Total subject IDs: {len(subject_ids)}")

    unique_subject_ids = np.unique(subject_ids)
    print(f"Unique subject IDs ({len(unique_subject_ids)}):")
    print(unique_subject_ids)

    print("\nFirst 20 subject IDs:")
    print(subject_ids[:20]) # there are multiple segment per subject... make sense
else:
    print("SubjectIDs key not found in the file.")


# add BMI and BMI category data in the demographic file

In [ ]:
# Paths - IMPORTANT: Use the original batch directory
save_dir = '/content/drive/MyDrive/CAS_FinalProject/CAS_FinalProject_RAW/Vital1'
output_dir = '/content/drive/MyDrive/CAS_FinalProject/Vital_With_BMI1' ########################## make sure this is BMI1
os.makedirs(output_dir, exist_ok=True)

def calculate_bmi(height_cm, weight_kg):
    """Calculate BMI from height (cm) and weight (kg)"""
    if np.isnan(height_cm) or np.isnan(weight_kg) or height_cm <= 0:
        return np.nan
    height_m = height_cm / 100.0
    bmi = weight_kg / (height_m ** 2)
    return bmi

def get_bmi_category(bmi):
    """Categorize BMI"""
    if np.isnan(bmi):
        return 'Unknown'
    elif bmi < 18.5:
        return 'Underweight'
    elif bmi < 25.0:
        return 'Normal Weight'
    elif bmi < 30.0:
        return 'Overweight'
    else:
        return 'Obese'

# Process each batch
batch_files = sorted(glob.glob(os.path.join(save_dir, 'PulseDB_Vital_Batch_*.npz')))

print(f"{'='*70}")
print(f"Adding BMI and BMI_Category to Demographics ")
print(f"{'='*70}\n")

total_segments = 0
valid_bmi_count = 0
bmi_category_counts = {'Underweight': 0, 'Normal Weight': 0, 'Overweight': 0, 'Obese': 0, 'Unknown': 0}

for idx, batch_file in enumerate(batch_files):
    batch_num = int(os.path.basename(batch_file).split('_')[-1].replace('.npz', ''))

    print(f"Processing Batch {batch_num:04d}...", end=' ')

    # Load batch - ONLY LOAD, DON'T PROCESS SIGNALS
    data = np.load(batch_file, allow_pickle=True)
    signals = data['Signals']                    # Keep as-is
    demographics = data['Demographics']          # Modify only this
    subject_ids = data['SubjectIDs']            # Keep as-is

    n_segments = demographics.shape[0]
    total_segments += n_segments

    # Calculate BMI and BMI_Category for each segment
    bmi_values = np.zeros(n_segments)
    bmi_categories = np.empty(n_segments, dtype=object)
    bmi_category_codes = np.zeros(n_segments, dtype=int)

    for seg_idx in range(n_segments):
        # Extract height and weight from original demographics
        # Original format: [Age, Gender, Height(cm), Weight(kg)]
        height_cm = demographics[seg_idx, 2]
        weight_kg = demographics[seg_idx, 3]

        # Calculate BMI
        bmi = calculate_bmi(height_cm, weight_kg)
        bmi_values[seg_idx] = bmi

        # Get BMI category
        bmi_category = get_bmi_category(bmi)
        bmi_categories[seg_idx] = bmi_category

        # Encode BMI_Category as numeric
        bmi_category_codes[seg_idx] = (
            0 if bmi_category == 'Unknown' else
            1 if bmi_category == 'Underweight' else
            2 if bmi_category == 'Normal Weight' else
            3 if bmi_category == 'Overweight' else
            4  # Obese
        )

        # Update statistics
        if not np.isnan(bmi):
            valid_bmi_count += 1
        bmi_category_counts[bmi_category] += 1

    # CREATE NEW DEMOGRAPHICS WITH BMI ADDED
    # New format: [Age, Gender, Height, Weight, BMI, BMI_Category_Code]
    demographics_with_bmi = np.column_stack((
        demographics,              # Original 4 columns
        bmi_values,               # Add BMI
        bmi_category_codes        # Add BMI_Category_Code
    ))

    # Save batch with ONLY demographics modified
    output_path = os.path.join(output_dir, f'PulseDB_Vital_Batch_{batch_num:04d}.npz')
    np.savez_compressed(output_path,
                        Signals=signals,              # ✅ UNCHANGED
                        Demographics=demographics_with_bmi,  # ✅ MODIFIED
                        SubjectIDs=subject_ids,       # ✅ UNCHANGED
                        BMI_Categories=bmi_categories)  # ✅ NEW (optional, for reference)

    print(f"✅ Saved ({n_segments} segments)")

    # Clean up
    data.close()

print(f"\n{'='*70}")
print(f"✅ BMI Addition Complete!")
print(f"{'='*70}")
print(f"Total segments processed: {total_segments:,}")
print(f"Valid BMI values: {valid_bmi_count:,} ({100*valid_bmi_count/total_segments:.1f}%)")

print(f"\nBMI Category Distribution:")
for category, count in bmi_category_counts.items():
    percentage = 100 * count / total_segments if total_segments > 0 else 0
    print(f"  {category:20s}: {count:>6,} ({percentage:>5.1f}%)")

print(f"\nOutput batches saved to: {output_dir}")
print(f"Demographics format: [Age, Gender, Height(cm), Weight(kg), BMI, BMI_Category_Code]")
print(f"BMI_Category_Code: 1=Underweight, 2=Normal, 3=Overweight, 4=Obese, 0=Unknown")

print(f"{'='*70}")

In [ ]:
# Path to batches with BMI
output_dir = '/content/drive/MyDrive/CAS_FinalProject/Vital_With_BMI1'

# Load all batches and collect BMI data
batch_files = sorted(glob.glob(os.path.join(output_dir, 'PulseDB_Vital_Batch_*.npz')))

print(f"{'='*70}")
print(f"Collecting BMI Data from {len(batch_files)} batches...")
print(f"{'='*70}\n")

all_bmi_data = []

for batch_idx, batch_file in enumerate(batch_files):
    batch_num = int(os.path.basename(batch_file).split('_')[-1].replace('.npz', ''))

    # Load batch
    data = np.load(batch_file, allow_pickle=True)
    demographics = data['Demographics']
    bmi_categories = data['BMI_Categories']
    subject_ids = data['SubjectIDs']

    # Extract BMI data (column 4) and category (column 5)
    for seg_idx in range(len(demographics)):
        bmi = demographics[seg_idx, 4]
        bmi_code = int(demographics[seg_idx, 5])
        bmi_category = bmi_categories[seg_idx]
        subject_id = subject_ids[seg_idx]

        all_bmi_data.append({
            'BMI': bmi,
            'BMI_Code': bmi_code,
            'BMI_Category': bmi_category,
            'Subject_ID': subject_id,
            'Batch': batch_num
        })

    data.close()

    if (batch_idx + 1) % 3 == 0:
        print(f"Processed {batch_idx + 1}/{len(batch_files)} batches...")

# Convert to DataFrame
df_bmi = pd.DataFrame(all_bmi_data)

# Get unique subjects (one row per subject)
df_unique = df_bmi.drop_duplicates(subset=['Subject_ID']).copy()

print(f"\n{'='*70}")
print(f"BMI DISTRIBUTION STATISTICS")
print(f"{'='*70}")
print(f"Total segments: {len(df_bmi):,}")
print(f"Unique subjects: {len(df_unique):,}")

print(f"\nBMI Statistics (All Segments):")
print(f"  Mean: {df_bmi['BMI'].mean():.2f}")
print(f"  Std:  {df_bmi['BMI'].std():.2f}")
print(f"  Min:  {df_bmi['BMI'].min():.2f}")
print(f"  Max:  {df_bmi['BMI'].max():.2f}")
print(f"  Median: {df_bmi['BMI'].median():.2f}")

print(f"\nBMI Statistics (Unique Subjects):")
print(f"  Mean: {df_unique['BMI'].mean():.2f}")
print(f"  Std:  {df_unique['BMI'].std():.2f}")
print(f"  Min:  {df_unique['BMI'].min():.2f}")
print(f"  Max:  {df_unique['BMI'].max():.2f}")
print(f"  Median: {df_unique['BMI'].median():.2f}")

print(f"\nBMI Category Distribution (All Segments):")
category_counts = df_bmi['BMI_Category'].value_counts()
for category, count in category_counts.items():
    percentage = 100 * count / len(df_bmi)
    print(f"  {category:20s}: {count:>8,} ({percentage:>5.1f}%)")

print(f"\nBMI Category Distribution (Unique Subjects):")
category_counts_unique = df_unique['BMI_Category'].value_counts()
for category, count in category_counts_unique.items():
    percentage = 100 * count / len(df_unique)
    print(f"  {category:20s}: {count:>4,} ({percentage:>5.1f}%)")

# ===== VISUALIZATIONS =====
fig = plt.figure(figsize=(18, 12))
gs = fig.add_gridspec(3, 3, hspace=0.35, wspace=0.3)

fig.suptitle('PulseDB_Vital Dataset - BMI Distribution Analysis',
             fontsize=16, fontweight='bold')

# ===== ROW 1: Histograms =====

# 1. BMI Histogram (All Segments)
ax1 = fig.add_subplot(gs[0, 0])
ax1.hist(df_bmi['BMI'].dropna(), bins=50, color='skyblue', edgecolor='black', alpha=0.7)
ax1.axvline(df_bmi['BMI'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {df_bmi["BMI"].mean():.1f}')
ax1.axvline(df_bmi['BMI'].median(), color='green', linestyle='--', linewidth=2, label=f'Median: {df_bmi["BMI"].median():.1f}')
ax1.set_xlabel('BMI', fontsize=11)
ax1.set_ylabel('Frequency (All Segments)', fontsize=11)
ax1.set_title('BMI Distribution - All Segments', fontsize=12, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. BMI Histogram (Unique Subjects)
ax2 = fig.add_subplot(gs[0, 1])
ax2.hist(df_unique['BMI'].dropna(), bins=40, color='lightcoral', edgecolor='black', alpha=0.7)
ax2.axvline(df_unique['BMI'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {df_unique["BMI"].mean():.1f}')
ax2.axvline(df_unique['BMI'].median(), color='green', linestyle='--', linewidth=2, label=f'Median: {df_unique["BMI"].median():.1f}')
ax2.set_xlabel('BMI', fontsize=11)
ax2.set_ylabel('Frequency (Unique Subjects)', fontsize=11)
ax2.set_title('BMI Distribution - Unique Subjects', fontsize=12, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. BMI with Category Regions
ax3 = fig.add_subplot(gs[0, 2])
ax3.hist(df_unique['BMI'].dropna(), bins=40, color='lightcoral', edgecolor='black', alpha=0.7)
# Add category region lines
ax3.axvline(18.5, color='blue', linestyle='-', linewidth=2, alpha=0.7, label='Underweight')
ax3.axvline(25.0, color='green', linestyle='-', linewidth=2, alpha=0.7, label='Normal')
ax3.axvline(30.0, color='orange', linestyle='-', linewidth=2, alpha=0.7, label='Overweight')
ax3.axvspan(30, df_unique['BMI'].max() + 5, alpha=0.2, color='red', label='Obese')
ax3.set_xlabel('BMI', fontsize=11)
ax3.set_ylabel('Frequency', fontsize=11)
ax3.set_title('BMI Categories Regions', fontsize=12, fontweight='bold')
ax3.legend(fontsize=9)
ax3.grid(True, alpha=0.3)

# ===== ROW 2: Category Distributions =====

# 4. BMI Categories - Bar Chart (All Segments)
ax4 = fig.add_subplot(gs[1, 0])
category_order = ['Underweight', 'Normal Weight', 'Overweight', 'Obese', 'Unknown']
counts_all = [df_bmi[df_bmi['BMI_Category'] == cat].shape[0] for cat in category_order]
colors_cat = ['lightblue', 'lightgreen', 'orange', 'red', 'gray']
bars = ax4.bar(category_order, counts_all, color=colors_cat, edgecolor='black', alpha=0.7)
ax4.set_ylabel('Count (All Segments)', fontsize=11)
ax4.set_title('BMI Categories - All Segments', fontsize=12, fontweight='bold')
ax4.tick_params(axis='x', rotation=45)
# Add value labels on bars
for bar, count in zip(bars, counts_all):
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(count):,}', ha='center', va='bottom', fontsize=9)
ax4.grid(True, alpha=0.3, axis='y')

# 5. BMI Categories - Bar Chart (Unique Subjects)
ax5 = fig.add_subplot(gs[1, 1])
counts_unique = [df_unique[df_unique['BMI_Category'] == cat].shape[0] for cat in category_order]
bars = ax5.bar(category_order, counts_unique, color=colors_cat, edgecolor='black', alpha=0.7)
ax5.set_ylabel('Count (Unique Subjects)', fontsize=11)
ax5.set_title('BMI Categories - Unique Subjects', fontsize=12, fontweight='bold')
ax5.tick_params(axis='x', rotation=45)
# Add value labels on bars
for bar, count in zip(bars, counts_unique):
    height = bar.get_height()
    ax5.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(count)}', ha='center', va='bottom', fontsize=9)
ax5.grid(True, alpha=0.3, axis='y')

# 6. BMI Categories - Pie Chart (Unique Subjects)
ax6 = fig.add_subplot(gs[1, 2])
# Filter out Unknown for cleaner pie chart
df_unique_known = df_unique[df_unique['BMI_Category'] != 'Unknown']
pie_counts = df_unique_known['BMI_Category'].value_counts()
pie_data = [pie_counts.get(cat, 0) for cat in ['Underweight', 'Normal Weight', 'Overweight', 'Obese']]
wedges, texts, autotexts = ax6.pie(pie_data,
                                     labels=['Underweight', 'Normal Weight', 'Overweight', 'Obese'],
                                     colors=['lightblue', 'lightgreen', 'orange', 'red'],
                                     autopct='%1.1f%%',
                                     startangle=90)
for autotext in autotexts:
    autotext.set_color('black')
    autotext.set_fontweight('bold')
ax6.set_title('BMI Categories Distribution\n(Unique Subjects, Known BMI)', fontsize=12, fontweight='bold')

# ===== ROW 3: Box Plots and Statistics =====

# 7. Box Plot by Category (All Segments)
ax7 = fig.add_subplot(gs[2, 0])
data_by_cat = [df_bmi[df_bmi['BMI_Category'] == cat]['BMI'].dropna().values for cat in category_order if cat != 'Unknown']
bp = ax7.boxplot(data_by_cat, labels=['Underweight', 'Normal', 'Overweight', 'Obese'],
                 patch_artist=True, widths=0.6)
for patch, color in zip(bp['boxes'], colors_cat[:4]):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)
ax7.set_ylabel('BMI', fontsize=11)
ax7.set_title('BMI Distribution by Category\n(All Segments)', fontsize=12, fontweight='bold')
ax7.grid(True, alpha=0.3, axis='y')

# 8. Box Plot by Category (Unique Subjects)
ax8 = fig.add_subplot(gs[2, 1])
data_by_cat_unique = [df_unique[df_unique['BMI_Category'] == cat]['BMI'].dropna().values for cat in category_order if cat != 'Unknown']
bp = ax8.boxplot(data_by_cat_unique, labels=['Underweight', 'Normal', 'Overweight', 'Obese'],
                 patch_artist=True, widths=0.6)
for patch, color in zip(bp['boxes'], colors_cat[:4]):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)
ax8.set_ylabel('BMI', fontsize=11)
ax8.set_title('BMI Distribution by Category\n(Unique Subjects)', fontsize=12, fontweight='bold')
ax8.grid(True, alpha=0.3, axis='y')

# 9. Summary Statistics Table
ax9 = fig.add_subplot(gs[2, 2])
ax9.axis('off')

# Create summary text
summary_text = f"""
DATASET SUMMARY

Total Segments: {len(df_bmi):,}
Unique Subjects: {len(df_unique):,}

BMI STATISTICS (Unique Subjects):
  Mean: {df_unique['BMI'].mean():.2f}
  Std: {df_unique['BMI'].std():.2f}
  Min: {df_unique['BMI'].min():.2f}
  Max: {df_unique['BMI'].max():.2f}

CATEGORY BREAKDOWN:
  Underweight: {counts_unique[0]:>4} ({100*counts_unique[0]/len(df_unique):>5.1f}%)
  Normal: {counts_unique[1]:>4} ({100*counts_unique[1]/len(df_unique):>5.1f}%)
  Overweight: {counts_unique[2]:>4} ({100*counts_unique[2]/len(df_unique):>5.1f}%)
  Obese: {counts_unique[3]:>4} ({100*counts_unique[3]/len(df_unique):>5.1f}%)
"""

ax9.text(0.1, 0.5, summary_text, fontsize=10, verticalalignment='center',
        fontfamily='monospace', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.savefig('bmi_distribution_analysis.png', dpi=200, bbox_inches='tight')
plt.show()

print(f"\n✅ Visualization saved as 'bmi_distribution_analysis.png'")
print(f"{'='*70}")

In [ ]:
# check data
# Correct path
save_dir = '/content/drive/MyDrive/CAS_FinalProject/Vital_With_BMI1'###################### check BMI1

def load_batch(batch_num):
    """Load a specific batch file"""
    batch_path = os.path.join(save_dir, f'PulseDB_Vital_Batch_{batch_num:04d}.npz')
    if os.path.exists(batch_path):
        data = np.load(batch_path, allow_pickle=True)
        return data['Signals'], data['Demographics'], data['SubjectIDs']
    return None, None, None


def iterate_batches():
    """Generator to iterate through batches without loading all into memory"""
    batch_files = sorted(glob.glob(os.path.join(save_dir, 'PulseDB_Vital_Batch_*.npz')))

    for batch_file in batch_files:
        data = np.load(batch_file, allow_pickle=True)
        yield data['Signals'], data['Demographics'], data['SubjectIDs']
        data.close()  # Free memory


def get_dataset_info():
    """Get info about the complete dataset without loading it all"""
    batch_files = sorted(glob.glob(os.path.join(save_dir, 'PulseDB_Vital_Batch_*.npz')))

    print(f"{'='*60}")
    print("PulseDB_Vital Dataset Information")
    print(f"{'='*60}")
    print(f"Total batch files: {len(batch_files)}")

    total_segments = 0
    total_subjects = set()

    # Collect demographics for statistics
    all_ages = []
    all_genders = []
    all_heights = []
    all_weights = []

    for idx, batch_file in enumerate(batch_files):
        file_size_mb = os.path.getsize(batch_file) / (1024**2)

        data = np.load(batch_file, allow_pickle=True)
        n_segments = data['Signals'].shape[0]
        signal_length = data['Signals'].shape[2]
        subjects = set(data['SubjectIDs'])

        # Collect demographics
        all_ages.extend(data['Demographics'][:, 0])
        all_genders.extend(data['Demographics'][:, 1])
        all_heights.extend(data['Demographics'][:, 2])
        all_weights.extend(data['Demographics'][:, 3])

        total_segments += n_segments
        total_subjects.update(subjects)

        print(f"  Batch {idx:02d}: {n_segments:>6,} segments, {len(subjects):>3} subjects, {file_size_mb:>6.1f} MB")

        data.close()

    # Convert to arrays for statistics
    all_ages = np.array(all_ages)
    all_genders = np.array(all_genders)
    all_heights = np.array(all_heights)
    all_weights = np.array(all_weights)

    total_size_mb = sum([os.path.getsize(f) / (1024**2) for f in batch_files])

    print(f"\n{'='*60}")
    print(f"Total segments: {total_segments:,}")
    print(f"Total unique subjects: {len(total_subjects)}")
    print(f"Samples per segment: {signal_length} (10 seconds @ {signal_length/10:.0f} Hz)")
    print(f"Total dataset size: {total_size_mb:.1f} MB ({total_size_mb/1024:.2f} GB)")

    print(f"\n{'='*60}")
    print("Demographics Summary")
    print(f"{'='*60}")
    print(f"Age:")
    print(f"  Mean: {all_ages.mean():.1f} years")
    print(f"  Std:  {all_ages.std():.1f} years")
    print(f"  Range: {all_ages.min():.0f} - {all_ages.max():.0f} years")

    print(f"\nGender:")
    n_male = (all_genders == 1).sum()
    n_female = (all_genders == 0).sum()
    print(f"  Male:   {n_male:,} ({100*n_male/len(all_genders):.1f}%)")
    print(f"  Female: {n_female:,} ({100*n_female/len(all_genders):.1f}%)")

    print(f"\nHeight:")
    print(f"  Mean: {np.nanmean(all_heights):.1f} cm")
    print(f"  Std:  {np.nanstd(all_heights):.1f} cm")

    print(f"\nWeight:")
    print(f"  Mean: {np.nanmean(all_weights):.1f} kg")
    print(f"  Std:  {np.nanstd(all_weights):.1f} kg")
    print(f"{'='*60}")

    return {
        'n_batches': len(batch_files),
        'total_segments': total_segments,
        'n_subjects': len(total_subjects),
        'signal_length': signal_length,
        'batch_files': batch_files
    }


# Run the analysis
print("\nAnalyzing your PulseDB_Vital dataset...\n")
info = get_dataset_info()

# Example: Load and use first batch
print(f"\n{'='*60}")
print("Example: Working with Batch 0")
print(f"{'='*60}")

signals, demographics, subject_ids = load_batch(0)

if signals is not None:
    print(f"Loaded Batch 0:")
    print(f"  Signals shape: {signals.shape}")
    print(f"    - signals[:, 0, :] = ECG")
    print(f"    - signals[:, 1, :] = PPG")
    print(f"    - signals[:, 2, :] = Time")
    print(f"  Demographics shape: {demographics.shape}")
    print(f"    - demographics[:, 0] = Age")
    print(f"    - demographics[:, 1] = Gender (1=M, 0=F)")
    print(f"    - demographics[:, 2] = Height (cm)")
    print(f"    - demographics[:, 3] = Weight (kg)")
    print(f"  Number of subjects: {len(np.unique(subject_ids))}")

    # Extract ECG and PPG
    ecg_batch0 = signals[:, 0, :]
    ppg_batch0 = signals[:, 1, :]

    print(f"\n  ECG shape: {ecg_batch0.shape}")
    print(f"  PPG shape: {ppg_batch0.shape}")


# Create a new subset containing 589 subjects with 150 from Underweight and Normal, 211 from overweight and 89 obeses (that is the max)

In [ ]:
save_dir = '/content/drive/MyDrive/CAS_FinalProject/Vital_With_BMI1'
output_dir_balanced = '/content/drive/MyDrive/CAS_FinalProject/Vital_With_BMI1_balanced'
os.makedirs(output_dir_balanced, exist_ok=True)

# NEW: Adjusted balanced counts per BMI category (total 600 subjects)
subjects_per_category = {
    'Underweight': 150,
    'Normal Weight': 150,
    'Overweight': 211,
    'Obese': 89
}

target_bmi_categories = ['Underweight', 'Normal Weight', 'Overweight', 'Obese']

# Track subjects selected for each category
selected_subjects_per_category = {cat: set() for cat in target_bmi_categories}

# Map from subject to all its indices and BMI categories
subject_info = defaultdict(lambda: {'indices': [], 'bmi_cats': []})

# ---------------------------------------------------
# Step 1: Aggregate subject info across batches
# ---------------------------------------------------
batch_files = sorted(glob.glob(os.path.join(save_dir, 'PulseDB_Vital_Batch_*.npz')))

print(f"{'='*70}")
print("Aggregating subject info and BMI categories from batches")
print(f"{'='*70}")

for batch_file in batch_files:
    data = np.load(batch_file, allow_pickle=True)
    demographics = data['Demographics']
    bmi_categories = data['BMI_Categories']
    subject_ids = data['SubjectIDs']
    n_segments = len(subject_ids)

    for i in range(n_segments):
        subj = subject_ids[i]
        subject_info[subj]['indices'].append((batch_file, i))
        subject_info[subj]['bmi_cats'].append(bmi_categories[i])

    data.close()

print(f"Total unique subjects found: {len(subject_info)}")

# ---------------------------------------------------
# Step 2: Determine each subject's BMI category by mode
# ---------------------------------------------------
subject_bmi_category = {}

for subj, info in subject_info.items():
    bmi_cats = info['bmi_cats']
    # Get the most common BMI category for this subject
    counts = defaultdict(int)
    for cat in bmi_cats:
        counts[cat] += 1
    mode_cat = max(counts, key=counts.get)
    subject_bmi_category[subj] = mode_cat

# ---------------------------------------------------
# Step 3: Select subjects for balanced dataset (NEW COUNTS)
# ---------------------------------------------------
total_selected = 0
for cat in target_bmi_categories:
    # Collect all subjects with this BMI category
    subjects_in_cat = [s for s,c in subject_bmi_category.items() if c == cat]
    needed = subjects_per_category[cat]

    print(f"{cat}: found {len(subjects_in_cat)} subjects, selecting {needed}")

    if len(subjects_in_cat) < needed:
        print(f"  WARNING: Only {len(subjects_in_cat)} available, using all")
        sampled = subjects_in_cat
    else:
        # Randomly sample without replacement
        sampled = np.random.choice(subjects_in_cat, size=needed, replace=False)

    selected_subjects_per_category[cat].update(sampled)
    total_selected += len(sampled)

selected_subjects_all = set()
for cat in target_bmi_categories:
    selected_subjects_all.update(selected_subjects_per_category[cat])

print(f"Total selected subjects for balanced dataset: {total_selected}")

# ---------------------------------------------------
# Step 4: Extract segments for selected subjects and save in batches of 50 subjects
# ---------------------------------------------------
print(f"{'='*70}")
print(f"Extracting segments and saving balanced dataset in batches of 10 subjects")
print(f"{'='*70}")

# Prepare container for accumulating batch data before saving
batch_size_subjects = 50
current_batch_subjects = []
current_signals = []
current_demographics = []
current_subject_ids = []
current_bmi_categories = []

def save_batch(batch_idx):
    out_path = os.path.join(output_dir_balanced, f'Balanced_PulseDB_Vital_Batch_{batch_idx:04d}.npz')
    np.savez_compressed(
        out_path,
        Signals=np.array(current_signals),
        Demographics=np.array(current_demographics),
        SubjectIDs=np.array(current_subject_ids),
        BMI_Categories=np.array(current_bmi_categories)
    )
    print(f"Saved batch {batch_idx:04d}: {len(current_signals)} segments, {len(set(current_subject_ids))} subjects")

batch_idx = 1

for cat in target_bmi_categories:
    for subj in list(selected_subjects_per_category[cat]):
        subj_indices = subject_info[subj]['indices']
        # Group indices by batch file for efficiency
        batch_file_to_indices = defaultdict(list)
        for fpath, idx_seg in subj_indices:
            batch_file_to_indices[fpath].append(idx_seg)

        for fpath, indices in batch_file_to_indices.items():
            data = np.load(fpath, allow_pickle=True)
            signals = data['Signals'][indices]
            demographics = data['Demographics'][indices]
            subject_ids = data['SubjectIDs'][indices]
            bmi_cats = data['BMI_Categories'][indices]
            data.close()

            # Append to current batch containers
            current_signals.extend(signals)
            current_demographics.extend(demographics)
            current_subject_ids.extend(subject_ids)
            current_bmi_categories.extend(bmi_cats)

        current_batch_subjects.append(subj)

        # Save batch if accumulated subjects reach batch_size_subjects
        if len(current_batch_subjects) == batch_size_subjects:
            save_batch(batch_idx)
            batch_idx += 1
            current_batch_subjects.clear()
            current_signals.clear()
            current_demographics.clear()
            current_subject_ids.clear()
            current_bmi_categories.clear()

# Save remaining if any
if current_batch_subjects:
    save_batch(batch_idx)

print(f"{'='*70}")
print("Balanced dataset creation complete!")
print(f"Final counts: 150 Underweight + 150 Normal + 211 Overweight + 89 Obese = {total_selected} total subjects")
print(f"Data saved under: {output_dir_balanced}")
print(f"{'='*70}")


# create a subset containing Normal only

In [ ]:


save_dir = '/content/drive/MyDrive/CAS_FinalProject/Vital_With_BMI1'
output_dir_balanced = '/content/drive/MyDrive/CAS_FinalProject/Vital_With_BMI1_Normal'
os.makedirs(output_dir_balanced, exist_ok=True)

# ONLY Normal Weight patients: 600 subjects
subjects_per_category = {'Normal Weight': 600}
target_bmi_categories = ['Normal Weight']  # Only Normal Weight

# Track subjects selected
selected_subjects_per_category = {cat: set() for cat in target_bmi_categories}

# Map from subject to all its indices and BMI categories
subject_info = defaultdict(lambda: {'indices': [], 'bmi_cats': []})

# ---------------------------------------------------
# Step 1: Aggregate subject info across batches
# ---------------------------------------------------
batch_files = sorted(glob.glob(os.path.join(save_dir, 'PulseDB_Vital_Batch_*.npz')))

print(f"{'='*70}")
print("Aggregating subject info - Normal Weight patients only")
print(f"{'='*70}")

for batch_file in batch_files:
    data = np.load(batch_file, allow_pickle=True)
    demographics = data['Demographics']
    bmi_categories = data['BMI_Categories']
    subject_ids = data['SubjectIDs']
    n_segments = len(subject_ids)

    for i in range(n_segments):
        subj = subject_ids[i]
        subject_info[subj]['indices'].append((batch_file, i))
        subject_info[subj]['bmi_cats'].append(bmi_categories[i])

    data.close()

print(f"Total unique subjects found: {len(subject_info)}")

# ---------------------------------------------------
# Step 2: Determine each subject's BMI category by mode
# ---------------------------------------------------
subject_bmi_category = {}

for subj, info in subject_info.items():
    bmi_cats = info['bmi_cats']
    # Get the most common BMI category for this subject
    counts = defaultdict(int)
    for cat in bmi_cats:
        counts[cat] += 1
    mode_cat = max(counts, key=counts.get)
    subject_bmi_category[subj] = mode_cat

# ---------------------------------------------------
# Step 3: Select 600 Normal Weight subjects
# ---------------------------------------------------
total_selected = 0
for cat in target_bmi_categories:
    # Collect all Normal Weight subjects
    subjects_in_cat = [s for s,c in subject_bmi_category.items() if c == cat]
    needed = subjects_per_category[cat]

    print(f"{cat}: found {len(subjects_in_cat)} subjects, selecting {needed}")

    if len(subjects_in_cat) < needed:
        print(f"  WARNING: Only {len(subjects_in_cat)} available, using all")
        sampled = subjects_in_cat
    else:
        # Randomly sample 600 Normal Weight subjects
        sampled = np.random.choice(subjects_in_cat, size=needed, replace=False)

    selected_subjects_per_category[cat].update(sampled)
    total_selected += len(sampled)

print(f"Total Normal Weight subjects selected: {total_selected}")

# ---------------------------------------------------
# Step 4: Extract segments and save in batches of 50 subjects
# ---------------------------------------------------
print(f"{'='*70}")
print(f"Extracting Normal Weight segments - batches of 50 subjects")
print(f"{'='*70}")

# Prepare container for accumulating batch data
batch_size_subjects = 50
current_batch_subjects = []
current_signals = []
current_demographics = []
current_subject_ids = []
current_bmi_categories = []

def save_batch(batch_idx):
    out_path = os.path.join(output_dir_balanced, f'Normal_PulseDB_Vital_Batch_{batch_idx:04d}.npz')
    np.savez_compressed(
        out_path,
        Signals=np.array(current_signals),
        Demographics=np.array(current_demographics),
        SubjectIDs=np.array(current_subject_ids),
        BMI_Categories=np.array(current_bmi_categories)
    )
    print(f"Saved batch {batch_idx:04d}: {len(current_signals)} segments, {len(set(current_subject_ids))} subjects")

batch_idx = 1

# Process only Normal Weight selected subjects
for subj in list(selected_subjects_per_category['Normal Weight']):
    subj_indices = subject_info[subj]['indices']
    # Group indices by batch file for efficiency
    batch_file_to_indices = defaultdict(list)
    for fpath, idx_seg in subj_indices:
        batch_file_to_indices[fpath].append(idx_seg)

    for fpath, indices in batch_file_to_indices.items():
        data = np.load(fpath, allow_pickle=True)
        signals = data['Signals'][indices]
        demographics = data['Demographics'][indices]
        subject_ids = data['SubjectIDs'][indices]
        bmi_cats = data['BMI_Categories'][indices]
        data.close()

        # Append to current batch containers
        current_signals.extend(signals)
        current_demographics.extend(demographics)
        current_subject_ids.extend(subject_ids)
        current_bmi_categories.extend(bmi_cats)

    current_batch_subjects.append(subj)

    # Save batch if accumulated 50 subjects
    if len(current_batch_subjects) == batch_size_subjects:
        save_batch(batch_idx)
        batch_idx += 1
        current_batch_subjects.clear()
        current_signals.clear()
        current_demographics.clear()
        current_subject_ids.clear()
        current_bmi_categories.clear()

# Save remaining subjects
if current_batch_subjects:
    save_batch(batch_idx)

print(f"{'='*70}")
print("Normal Weight dataset creation complete!")
print(f"Final count: {total_selected} Normal Weight subjects")
print(f"Data saved under: {output_dir_balanced}")
print(f"{'='*70}")


# Normalize and split each segment into a 4 sec windows with 2 sec overlap (do not rerun)

In [ ]:
# normalize each segment individually per segment

save_dir = '/content/drive/MyDrive/CAS_FinalProject/Vital_With_BMI1_balanced'################# make sure it is Vital_With_BMI1
normalized_dir = '/content/drive/MyDrive/CAS_FinalProject/Vital_Normalized1_balanced'############### Vital_Normalized1
os.makedirs(normalized_dir, exist_ok=True)

def normalize_to_range(signal, min_val=-1, max_val=1):
    """
    Normalize signal to specified range [min_val, max_val]
    Default: [-1, 1]
    """
    sig_min = signal.min()
    sig_max = signal.max()

    # Avoid division by zero
    if sig_max - sig_min == 0:
        return np.zeros_like(signal)

    # Min-Max normalization formula
    normalized = (signal - sig_min) / (sig_max - sig_min)  # [0, 1]
    normalized = normalized * (max_val - min_val) + min_val  # [min_val, max_val]

    return normalized


# Process each batch
batch_files = sorted(glob.glob(os.path.join(save_dir, 'Balanced_PulseDB_Vital_Batch_*.npz')))

print(f"Normalizing {len(batch_files)} batches...\n")

for idx, batch_file in enumerate(batch_files):
    batch_num = int(os.path.basename(batch_file).split('_')[-1].replace('.npz', ''))

    print(f"Processing Batch {batch_num:04d}...", end=' ')

    # Load batch
    data = np.load(batch_file, allow_pickle=True)
    signals = data['Signals']
    demographics = data['Demographics']
    subject_ids = data['SubjectIDs']

    # Get original shape
    n_segments, n_channels, signal_length = signals.shape

    # Create normalized copy
    signals_normalized = np.zeros_like(signals)

    # Normalize each segment independently
    for seg_idx in range(n_segments):
        # Normalize ECG (channel 0)
        signals_normalized[seg_idx, 0, :] = normalize_to_range(signals[seg_idx, 0, :])

        # Normalize PPG (channel 1)
        signals_normalized[seg_idx, 1, :] = normalize_to_range(signals[seg_idx, 1, :])

        # Keep Time (channel 2) as is (don't normalize time)
        signals_normalized[seg_idx, 2, :] = signals[seg_idx, 2, :]

    # Save normalized batch
    output_path = os.path.join(normalized_dir, f'PulseDB_Vital_Batch_{batch_num:04d}_Normalized.npz')
    np.savez_compressed(output_path,
                        Signals=signals_normalized,
                        Signals_Original=signals,  # Keep original for reference
                        Demographics=demographics,
                        SubjectIDs=subject_ids)

    print(f"✅ Saved ({n_segments} segments)")

    # Clean up
    data.close()

print(f"\n{'='*60}")
print(f"✅ Normalization complete!")
print(f"Normalized batches saved to: {normalized_dir}")
print(f"{'='*60}")


In [ ]:
## check data: make sure nothing has happen
# Correct path
save_dir = '/content/drive/MyDrive/CAS_FinalProject/Vital_Normalized1_balanced' ##################### make sure it is Vital_Normalized1

def load_batch(batch_num):
    """Load a specific batch file"""
    batch_path = os.path.join(save_dir, f'PulseDB_Vital_Batch_{batch_num:04d}.npz')
    if os.path.exists(batch_path):
        data = np.load(batch_path, allow_pickle=True)
        return data['Signals'], data['Demographics'], data['SubjectIDs']
    return None, None, None


def iterate_batches():
    """Generator to iterate through batches without loading all into memory"""
    batch_files = sorted(glob.glob(os.path.join(save_dir, 'PulseDB_Vital_Batch_*.npz')))

    for batch_file in batch_files:
        data = np.load(batch_file, allow_pickle=True)
        yield data['Signals'], data['Demographics'], data['SubjectIDs']
        data.close()  # Free memory


def get_dataset_info():
    """Get info about the complete dataset without loading it all"""
    batch_files = sorted(glob.glob(os.path.join(save_dir, 'PulseDB_Vital_Batch_*.npz')))

    print(f"{'='*60}")
    print("PulseDB_Vital Dataset Information")
    print(f"{'='*60}")
    print(f"Total batch files: {len(batch_files)}")

    total_segments = 0
    total_subjects = set()

    # Collect demographics for statistics
    all_ages = []
    all_genders = []
    all_heights = []
    all_weights = []

    for idx, batch_file in enumerate(batch_files):
        file_size_mb = os.path.getsize(batch_file) / (1024**2)

        data = np.load(batch_file, allow_pickle=True)
        n_segments = data['Signals'].shape[0]
        signal_length = data['Signals'].shape[2]
        subjects = set(data['SubjectIDs'])

        # Collect demographics
        all_ages.extend(data['Demographics'][:, 0])
        all_genders.extend(data['Demographics'][:, 1])
        all_heights.extend(data['Demographics'][:, 2])
        all_weights.extend(data['Demographics'][:, 3])

        total_segments += n_segments
        total_subjects.update(subjects)

        print(f"  Batch {idx:02d}: {n_segments:>6,} segments, {len(subjects):>3} subjects, {file_size_mb:>6.1f} MB")

        data.close()

    # Convert to arrays for statistics
    all_ages = np.array(all_ages)
    all_genders = np.array(all_genders)
    all_heights = np.array(all_heights)
    all_weights = np.array(all_weights)

    total_size_mb = sum([os.path.getsize(f) / (1024**2) for f in batch_files])

    print(f"\n{'='*60}")
    print(f"Total segments: {total_segments:,}")
    print(f"Total unique subjects: {len(total_subjects)}")
    print(f"Samples per segment: {signal_length} (10 seconds @ {signal_length/10:.0f} Hz)")
    print(f"Total dataset size: {total_size_mb:.1f} MB ({total_size_mb/1024:.2f} GB)")

    print(f"\n{'='*60}")
    print("Demographics Summary")
    print(f"{'='*60}")
    print(f"Age:")
    print(f"  Mean: {all_ages.mean():.1f} years")
    print(f"  Std:  {all_ages.std():.1f} years")
    print(f"  Range: {all_ages.min():.0f} - {all_ages.max():.0f} years")

    print(f"\nGender:")
    n_male = (all_genders == 1).sum()
    n_female = (all_genders == 0).sum()
    print(f"  Male:   {n_male:,} ({100*n_male/len(all_genders):.1f}%)")
    print(f"  Female: {n_female:,} ({100*n_female/len(all_genders):.1f}%)")

    print(f"\nHeight:")
    print(f"  Mean: {np.nanmean(all_heights):.1f} cm")
    print(f"  Std:  {np.nanstd(all_heights):.1f} cm")

    print(f"\nWeight:")
    print(f"  Mean: {np.nanmean(all_weights):.1f} kg")
    print(f"  Std:  {np.nanstd(all_weights):.1f} kg")
    print(f"{'='*60}")

    return {
        'n_batches': len(batch_files),
        'total_segments': total_segments,
        'n_subjects': len(total_subjects),
        'signal_length': signal_length,
        'batch_files': batch_files
    }


# Run the analysis
print("\nAnalyzing your PulseDB_Vital dataset...\n")
info = get_dataset_info()

# Example: Load and use first batch
print(f"\n{'='*60}")
print("Example: Working with Batch 0")
print(f"{'='*60}")

signals, demographics, subject_ids = load_batch(0)

if signals is not None:
    print(f"Loaded Batch 0:")
    print(f"  Signals shape: {signals.shape}")
    print(f"    - signals[:, 0, :] = ECG")
    print(f"    - signals[:, 1, :] = PPG")
    print(f"    - signals[:, 2, :] = Time")
    print(f"  Demographics shape: {demographics.shape}")
    print(f"    - demographics[:, 0] = Age")
    print(f"    - demographics[:, 1] = Gender (1=M, 0=F)")
    print(f"    - demographics[:, 2] = Height (cm)")
    print(f"    - demographics[:, 3] = Weight (kg)")
    print(f"  Number of subjects: {len(np.unique(subject_ids))}")

    # Extract ECG and PPG
    ecg_batch0 = signals[:, 0, :]
    ppg_batch0 = signals[:, 1, :]

    print(f"\n  ECG shape: {ecg_batch0.shape}")
    print(f"  PPG shape: {ppg_batch0.shape}")

 now create windows of 4 sec each with 2 sec overlapping

In [ ]:
# NEW: keep subject ID for traceability

# Paths
normalized_dir = '/content/drive/MyDrive/CAS_FinalProject/Vital_Normalized1_balanced' ################### make sure it is 1
windowed_dir = '/content/drive/MyDrive/CAS_FinalProject/Vital_Windowed1_balanced' ################### make sure it is 1
os.makedirs(windowed_dir, exist_ok=True)

# Parameters
SEGMENT_DURATION = 10  # seconds
WINDOW_DURATION = 4    # seconds
OVERLAP = 2            # seconds
STRIDE = WINDOW_DURATION - OVERLAP  # 2 seconds
SAMPLING_RATE = 125    # Hz (1250 samples / 10 seconds)

# Calculate samples
samples_per_window = int(WINDOW_DURATION * SAMPLING_RATE)  # 500 samples
stride_samples = int(STRIDE * SAMPLING_RATE)                # 250 samples

print(f"{'='*70}")
print(f"WINDOWING PARAMETERS")
print(f"{'='*70}")
print(f"Segment duration:     {SEGMENT_DURATION} seconds ({SEGMENT_DURATION * SAMPLING_RATE} samples)")
print(f"Window duration:      {WINDOW_DURATION} seconds ({samples_per_window} samples)")
print(f"Overlap:              {OVERLAP} seconds ({OVERLAP * SAMPLING_RATE} samples)")
print(f"Stride:               {STRIDE} seconds ({stride_samples} samples)")
print(f"Sampling rate:        {SAMPLING_RATE} Hz")
print(f"Windows per segment:  {(SEGMENT_DURATION - WINDOW_DURATION) // STRIDE + 1}")
print(f"{'='*70}\n")


def create_windows(signal, window_size, stride):
    """
    Create overlapping windows from a signal

    Parameters:
    -----------
    signal : numpy array of shape (signal_length,)
    window_size : int, number of samples per window
    stride : int, number of samples to shift between windows

    Returns:
    --------
    windows : numpy array of shape (n_windows, window_size)
    """
    signal_length = len(signal)
    n_windows = (signal_length - window_size) // stride + 1

    windows = np.zeros((n_windows, window_size))

    for i in range(n_windows):
        start_idx = i * stride
        end_idx = start_idx + window_size
        windows[i, :] = signal[start_idx:end_idx]

    return windows



# Process each batch
batch_files = sorted(glob.glob(os.path.join(normalized_dir, 'PulseDB_Vital_Batch_*_Normalized.npz')))

print(f"Processing {len(batch_files)} batches...\n")

total_original_segments = 0
total_windows_created = 0

for batch_idx, batch_file in enumerate(batch_files):
    batch_num = int(os.path.basename(batch_file).split('_')[-2])

    print(f"Processing Batch {batch_num:04d}...", end=' ')

    # Load batch
    data = np.load(batch_file, allow_pickle=True)
    signals = data['Signals']
    demographics = data['Demographics']
    subject_ids = data['SubjectIDs']

    n_segments = signals.shape[0]
    signal_length = signals.shape[2]

    # Lists to store windowed data
    all_ecg_windows = []
    all_ppg_windows = []
    all_window_demographics = []
    all_window_ids = []
    all_window_subject_ids = []

    # Process each segment
    for seg_idx in range(n_segments):
        # Extract ECG and PPG for this segment
        ecg_segment = signals[seg_idx, 0, :]
        ppg_segment = signals[seg_idx, 1, :]

        # Create windows
        ecg_windows = create_windows(ecg_segment, samples_per_window, stride_samples)
        ppg_windows = create_windows(ppg_segment, samples_per_window, stride_samples)

        n_windows = ecg_windows.shape[0]

        # Create unique window IDs
        # Format: B{batch}_S{segment}_W{window}
        # Example: B0000_S00123_W0, B0000_S00123_W1, etc.
        for win_idx in range(n_windows):
            # Unique window ID
            window_id = f"B{batch_num:04d}_S{seg_idx:05d}_W{win_idx}"

            all_ecg_windows.append(ecg_windows[win_idx, :])
            all_ppg_windows.append(ppg_windows[win_idx, :])
            all_window_demographics.append(demographics[seg_idx, :])
            all_window_ids.append(window_id)
            all_window_subject_ids.append(subject_ids[seg_idx])

    # Convert to numpy arrays
    all_ecg_windows = np.array(all_ecg_windows)
    all_ppg_windows = np.array(all_ppg_windows)
    all_window_demographics = np.array(all_window_demographics)
    all_window_ids = np.array(all_window_ids)
    all_window_subject_ids = np.array(all_window_subject_ids)

    # Save windowed batch
    output_path = os.path.join(windowed_dir, f'PulseDB_Vital_Batch_{batch_num:04d}_Windowed.npz')
    np.savez_compressed(output_path,
                        ECG_Windows=all_ecg_windows,
                        PPG_Windows=all_ppg_windows,
                        Demographics=all_window_demographics,
                        Window_IDs=all_window_ids,
                        SubjectIDs=all_window_subject_ids,
                        metadata={
                            'window_duration_sec': WINDOW_DURATION,
                            'overlap_sec': OVERLAP,
                            'stride_sec': STRIDE,
                            'sampling_rate_hz': SAMPLING_RATE,
                            'samples_per_window': samples_per_window,
                            'original_segments': n_segments,
                            'total_windows': len(all_window_ids)
                        })

    total_original_segments += n_segments
    total_windows_created += len(all_window_ids)

    print(f"✅ {n_segments:,} segments → {len(all_window_ids):,} windows")

    data.close()

print(f"\n{'='*70}")
print(f"WINDOWING COMPLETE")
print(f"{'='*70}")
print(f"Total original segments:  {total_original_segments:,}")
print(f"Total windows created:    {total_windows_created:,}")
print(f"Windows per segment:      {total_windows_created / total_original_segments:.1f}")
print(f"Windowed batches saved to: {windowed_dir}")
print(f"{'='*70}")


In [ ]:
# visualize the results
# Paths
normalized_dir = '/content/drive/MyDrive/CAS_FinalProject/Vital_Normalized'
windowed_dir = '/content/drive/MyDrive/CAS_FinalProject/Vital_Windowed'

# Load original batch (normalized)
original_batch = np.load(os.path.join(normalized_dir, 'PulseDB_Vital_Batch_0000_Normalized.npz'), allow_pickle=True)
signals_original = original_batch['Signals']
subject_ids_original = original_batch['SubjectIDs']

# Load windowed batch
windowed_batch = np.load(os.path.join(windowed_dir, 'PulseDB_Vital_Batch_0000_Windowed.npz'), allow_pickle=True)
ecg_windows = windowed_batch['ECG_Windows']
ppg_windows = windowed_batch['PPG_Windows']
window_ids = windowed_batch['Window_IDs']
metadata = windowed_batch['metadata'].item()

# Parameters
SAMPLING_RATE = 125
samples_per_window = metadata['samples_per_window']
stride_samples = int(metadata['stride_sec'] * SAMPLING_RATE)

# Select first segment (segment 0)
segment_idx = 0
subject_id = subject_ids_original[segment_idx]

# Extract original ECG and PPG for this segment
ecg_segment = signals_original[segment_idx, 0, :]  # 1250 samples
ppg_segment = signals_original[segment_idx, 1, :]  # 1250 samples

# Find all windows from this segment
segment_window_pattern = f"B0000_S{segment_idx:05d}_W"
segment_window_indices = [i for i, wid in enumerate(window_ids) if wid.startswith(segment_window_pattern)]

print(f"{'='*70}")
print(f"VISUALIZATION: Segment {segment_idx} Windowing (ECG & PPG)")
print(f"{'='*70}")
print(f"Subject ID: {subject_id}")
print(f"Original segment length: {len(ecg_segment)} samples (10 seconds)")
print(f"Number of windows: {len(segment_window_indices)}")
print(f"Window IDs: {[window_ids[i] for i in segment_window_indices]}")
print(f"{'='*70}\n")

# Time axes
time_original = np.arange(len(ecg_segment)) / SAMPLING_RATE
time_window = np.arange(samples_per_window) / SAMPLING_RATE

# Create comprehensive visualization
fig = plt.figure(figsize=(18, 14))
gs = fig.add_gridspec(6, 2, hspace=0.6, wspace=0.3) # Increased hspace

# Main title
fig.suptitle(f'ECG & PPG Windowing Visualization - Subject {subject_id} (Segment {segment_idx})',
             fontsize=16, fontweight='bold')

# ===== ROW 0: Original Segments =====
colors = ['red', 'orange', 'green', 'purple', 'brown']

# ECG Original
ax1 = fig.add_subplot(gs[0, 0])
ax1.plot(time_original, ecg_segment, 'b-', linewidth=1, label='Original ECG')

# Highlight window regions
for i, win_idx in enumerate(segment_window_indices):
    window_num = int(window_ids[win_idx].split('_W')[-1])
    start_sample = window_num * stride_samples
    end_sample = start_sample + samples_per_window

    start_time = start_sample / SAMPLING_RATE
    end_time = end_sample / SAMPLING_RATE

    ax1.axvspan(start_time, end_time, alpha=0.2, color=colors[i % len(colors)],
                label=f'W{window_num}')

ax1.set_xlabel('Time (seconds)', fontsize=11)
ax1.set_ylabel('Amplitude', fontsize=11)
ax1.set_title('Original ECG Segment (10s) with Window Regions', fontsize=12, fontweight='bold', pad=10) # Increased pad
ax1.grid(True, alpha=0.3)
ax1.legend(loc='upper right', fontsize=9)
ax1.set_xlim([0, 10])
ax1.set_ylim([-1.1, 1.1])

# PPG Original
ax2 = fig.add_subplot(gs[0, 1])
ax2.plot(time_original, ppg_segment, 'r-', linewidth=1, label='Original PPG')

# Highlight window regions
for i, win_idx in enumerate(segment_window_indices):
    window_num = int(window_ids[win_idx].split('_W')[-1])
    start_sample = window_num * stride_samples
    end_sample = start_sample + samples_per_window

    start_time = start_sample / SAMPLING_RATE
    end_time = end_sample / SAMPLING_RATE

    ax2.axvspan(start_time, end_time, alpha=0.2, color=colors[i % len(colors)],
                label=f'W{window_num}')

ax2.set_xlabel('Time (seconds)', fontsize=11)
ax2.set_ylabel('Amplitude', fontsize=11)
ax2.set_title('Original PPG Segment (10s) with Window Regions', fontsize=12, fontweight='bold', pad=10) # Increased pad
ax2.grid(True, alpha=0.3)
ax2.legend(loc='upper right', fontsize=9)
ax2.set_xlim([0, 10])
ax2.set_ylim([-1.1, 1.1])

# ===== ROWS 1-4: Individual Windows (ECG left, PPG right) =====
n_windows = len(segment_window_indices)

for i, win_idx in enumerate(segment_window_indices):
    if i >= 5:  # Limit to 5 windows
        break

    row = 1 + i

    ecg_win = ecg_windows[win_idx]
    ppg_win = ppg_windows[win_idx]
    window_id = window_ids[win_idx]
    window_num = int(window_id.split('_W')[-1])

    # ECG Window (left column)
    ax_ecg = fig.add_subplot(gs[row, 0])
    ax_ecg.plot(time_window, ecg_win, color=colors[i % len(colors)], linewidth=1.5)
    ax_ecg.set_xlabel('Time (seconds)', fontsize=10)
    ax_ecg.set_ylabel('Amplitude', fontsize=10)
    ax_ecg.set_title(f'ECG Window {window_num} (ID: {window_id})', fontsize=11, fontweight='bold', pad=5) # Increased pad
    ax_ecg.grid(True, alpha=0.3)
    ax_ecg.set_xlim([0, 4])
    ax_ecg.set_ylim([-1.1, 1.1])
    ax_ecg.axhline(y=0, color='k', linestyle='--', alpha=0.3)

    # Add range info
    ax_ecg.text(0.02, 0.98, f'Range: [{ecg_win.min():.2f}, {ecg_win.max():.2f}]',
               transform=ax_ecg.transAxes, fontsize=9,
               verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

    # PPG Window (right column)
    ax_ppg = fig.add_subplot(gs[row, 1])
    ax_ppg.plot(time_window, ppg_win, color=colors[i % len(colors)], linewidth=1.5)
    ax_ppg.set_xlabel('Time (seconds)', fontsize=10)
    ax_ppg.set_ylabel('Amplitude', fontsize=10)
    ax_ppg.set_title(f'PPG Window {window_num} (ID: {window_id})', fontsize=11, fontweight='bold', pad=5) # Increased pad
    ax_ppg.grid(True, alpha=0.3)
    ax_ppg.set_xlim([0, 4])
    ax_ppg.set_ylim([-1.1, 1.1])
    ax_ppg.axhline(y=0, color='k', linestyle='--', alpha=0.3)

    # Add range info
    ax_ppg.text(0.02, 0.98, f'Range: [{ppg_win.min():.2f}, {ppg_win.max():.2f}]',
               transform=ax_ppg.transAxes, fontsize=9,
               verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.savefig('segment_windowing_ecg_ppg_visualization.png', dpi=200, bbox_inches='tight')
plt.show()

print(f"\n✅ Visualization saved as 'segment_windowing_ecg_ppg_visualization.png'")

# Print detailed information
print(f"\n{'='*70}")
print(f"WINDOW DETAILS (ECG & PPG)")
print(f"{'='*70}")
for i, win_idx in enumerate(segment_window_indices):
    window_id = window_ids[win_idx]
    window_num = int(window_id.split('_W')[-1])
    start_time = window_num * metadata['stride_sec']
    end_time = start_time + metadata['window_duration_sec']

    ecg_win = ecg_windows[win_idx]
    ppg_win = ppg_windows[win_idx]

    print(f"\nWindow {window_num} (ID: {window_id}):")
    print(f"  Time range:      [{start_time:.1f}s - {end_time:.1f}s]")
    print(f"  ECG shape:       {ecg_win.shape}")
    print(f"  ECG range:       [{ecg_win.min():.3f}, {ecg_win.max():.3f}]")
    print(f"  ECG mean/std:    {ecg_win.mean():.3f} / {ecg_win.std():.3f}")
    print(f"  PPG shape:       {ppg_win.shape}")
    print(f"  PPG range:       [{ppg_win.min():.3f}, {ppg_win.max():.3f}]")
    print(f"  PPG mean/std:    {ppg_win.mean():.3f} / {ppg_win.std():.3f}")
    print(f"  ✅ Paired:       ECG and PPG have same window ID")

# Close files
original_batch.close()
windowed_batch.close()

print(f"\n{'='*70}")
print(f"✅ Visualization complete!")
print(f"{'='*70}")

# use the initial Normalized data and resample to 100Hz for ECG CPC

In [ ]:


save_dir = '/content/drive/MyDrive/CAS_FinalProject/Vital_With_BMI'
normalized_dir = '/content/drive/MyDrive/CAS_FinalProject/Vital_Normalized_Resampled_100Hz'
os.makedirs(normalized_dir, exist_ok=True)

def normalize_to_range(signal, min_val=-1, max_val=1):
    """
    Normalize signal to specified range [min_val, max_val]
    Default: [-1, 1]
    """
    sig_min = signal.min()
    sig_max = signal.max()

    # Avoid division by zero
    if sig_max - sig_min == 0:
        return np.zeros_like(signal)

    # Min-Max normalization formula
    normalized = (signal - sig_min) / (sig_max - sig_min)  # [0, 1]
    normalized = normalized * (max_val - min_val) + min_val  # [min_val, max_val]

    return normalized

# Process each batch
batch_files = sorted(glob.glob(os.path.join(save_dir, 'PulseDB_Vital_Batch_*.npz')))

print(f"Processing {len(batch_files)} batches...\n")

for idx, batch_file in enumerate(batch_files):
    batch_num = int(os.path.basename(batch_file).split('_')[-1].replace('.npz', ''))

    print(f"Processing Batch {batch_num:04d}...", end=' ')

    # Load batch
    data = np.load(batch_file, allow_pickle=True)
    signals = data['Signals']  # Shape: [n_segments, 3, original_length] @ 125 Hz
    demographics = data['Demographics']
    subject_ids = data['SubjectIDs']

    # Get original shape
    n_segments, n_channels, original_length = signals.shape
    print(f"Original: {n_segments} segs x {n_channels} ch x {original_length} samples")

    # Step 1: Normalize each segment individually
    signals_normalized = np.zeros_like(signals)
    for seg_idx in range(n_segments):
        # Normalize ECG (channel 0)
        signals_normalized[seg_idx, 0, :] = normalize_to_range(signals[seg_idx, 0, :])
        # Normalize PPG (channel 1)
        signals_normalized[seg_idx, 1, :] = normalize_to_range(signals[seg_idx, 1, :])
        # Keep Time (channel 2) as is
        signals_normalized[seg_idx, 2, :] = signals[seg_idx, 2, :]

    # Step 2: Resample to 100 Hz for 10 seconds (1000 samples)
    signals_resampled = np.zeros((n_segments, n_channels, 1000))  # Target: [n_seg, 3, 1000]

    for seg_idx in range(n_segments):
        for ch_idx in range(n_channels):
            # Resample each channel independently to 1000 samples (10 sec @ 100 Hz)
            signals_resampled[seg_idx, ch_idx, :] = resample(
                signals_normalized[seg_idx, ch_idx, :],
                1000,  # Target length for CPC input_size=1000
                axis=-1
            )

    # Save resampled batch (CPC-ready!)
    output_path = os.path.join(normalized_dir, f'PulseDB_Vital_Batch_{batch_num:04d}_Resampled100Hz.npz')
    np.savez_compressed(output_path,
                        Signals=signals_resampled,           # [n_seg, 3, 1000] @ 100 Hz
                        Signals_Normalized=signals_normalized,  # Keep normalized original
                        Signals_Original=signals,            # Keep raw original
                        Demographics=demographics,
                        SubjectIDs=subject_ids)

    print(f"✅ Saved ({n_segments} segments: {original_length}→1000 samples @100Hz)")

    # Clean up
    data.close()

print(f"\n{'='*70}")
print(f"✅ Resampling + Normalization COMPLETE!")
print(f"CPC-ready batches saved to: {normalized_dir}")
print(f"Format: [segments, 3_channels, 1000_samples] @ 100 Hz (10 sec)")
print(f"{'='*70}")
